In [19]:
import gower
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from scipy.optimize import linear_sum_assignment

In [20]:
og_df = pd.read_csv("datasets/heart_disease_uci.csv")
og_df.drop(columns=["id", "dataset"], inplace=True)
og_df = og_df.drop(og_df[og_df["num"] == 0].sample(frac=0.3).index)
og_df # this df still has "num" -> the target

,age,sex,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,63,Male,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,67,Male,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,67,Male,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,37,Male,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,41,Female,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914,46,Male,asymptomatic,134.0,310.0,False,normal,126.0,False,0.0,NaN,NaN,normal,2
915,54,Female,asymptomatic,127.0,333.0,True,st-t abnormality,154.0,False,0.0,NaN,NaN,NaN,1
917,55,Male,asymptomatic,122.0,223.0,True,st-t abnormality,100.0,False,0.0,NaN,NaN,fixed defect,2
918,58,Male,asymptomatic,NaN,385.0,True,lv hypertrophy,NaN,NaN,NaN,NaN,NaN,NaN,0


In [21]:
categorial_columns = ["sex", "cp", "fbs", "restecg", "exang", "slope", "thal"]
cont_columns = ["age", "trestbps", "chol", "thalch", "oldpeak", "ca"]

In [28]:
df = og_df.copy()
df.drop(columns="num", inplace=True)
df[categorial_columns] = df[categorial_columns].apply(LabelEncoder().fit_transform)
df[categorial_columns] = MinMaxScaler().fit_transform(df[categorial_columns])

df[cont_columns] = MinMaxScaler().fit_transform(df[cont_columns])
df = df.fillna(df.mean())
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal
0,0.714286,1.0,1.000000,0.725000,0.386401,0.5,0.000000,0.633803,0.0,0.556818,0.000000,0.000000,0.000000
1,0.795918,1.0,0.000000,0.800000,0.474295,0.0,0.000000,0.338028,0.5,0.465909,0.333333,1.000000,0.333333
2,0.795918,1.0,0.000000,0.600000,0.379768,0.0,0.000000,0.485915,0.5,0.590909,0.333333,0.666667,0.666667
3,0.183673,1.0,0.666667,0.650000,0.414594,0.0,0.333333,0.894366,0.0,0.693182,0.000000,0.000000,0.333333
4,0.265306,0.0,0.333333,0.650000,0.338308,0.0,0.000000,0.788732,0.0,0.454545,0.666667,0.000000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
914,0.367347,1.0,0.000000,0.670000,0.514096,0.0,0.333333,0.464789,0.0,0.295455,1.000000,0.248340,0.333333
915,0.530612,0.0,0.000000,0.635000,0.552239,0.5,0.666667,0.661972,0.0,0.295455,1.000000,0.248340,1.000000
917,0.551020,1.0,0.000000,0.610000,0.369818,0.5,0.666667,0.281690,0.0,0.295455,1.000000,0.248340,0.000000
918,0.612245,1.0,0.000000,0.659027,0.638474,0.5,0.000000,0.534525,1.0,0.402392,1.000000,0.248340,1.000000


In [30]:
class HeartDiseaseDataset(Dataset):
    def __init__(self, df):
        self.cat = torch.tensor(df[categorial_columns].values, dtype=torch.float)
        self.cont = torch.tensor(df[cont_columns].values, dtype=torch.float)

    def __getitem__(self, idx):
        return self.cat[idx], self.cont[idx]

    def __len__(self):
        return self.cat.shape[0]

In [31]:
dataset = HeartDiseaseDataset(df)
dataloader = DataLoader(dataset, batch_size=100, shuffle=True)
len(dataset)

797

In [34]:
class MixedTypeClusteringModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(len(df.columns), 9),
            torch.nn.Sigmoid(),
            torch.nn.Linear(9, 5),
            torch.nn.Sigmoid(),
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(5, 5),
            torch.nn.Sigmoid(),
            torch.nn.Linear(5, len(categorial_columns)),
            torch.nn.Sigmoid()
        )


    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


epochs = 100
lr = 0.001

model = MixedTypeClusteringModel()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for epoch in range(epochs):
    model.train()
    loss = 0

    for cat, cont in dataloader:
        optimizer.zero_grad()
        outputs = model(torch.cat((cat, cont), 1))
        train_loss = criterion(outputs,  cat)
        train_loss.backward()
        optimizer.step()
        loss += train_loss.item()

    loss = loss / len(dataloader)
    print("epoch: {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))

epoch: 1/100, loss = 0.158717
epoch: 2/100, loss = 0.155766
epoch: 3/100, loss = 0.152870
epoch: 4/100, loss = 0.150066
epoch: 5/100, loss = 0.147398
epoch: 6/100, loss = 0.144783
epoch: 7/100, loss = 0.142291
epoch: 8/100, loss = 0.139842
epoch: 9/100, loss = 0.137466
epoch: 10/100, loss = 0.135151
epoch: 11/100, loss = 0.132967
epoch: 12/100, loss = 0.130827
epoch: 13/100, loss = 0.128729
epoch: 14/100, loss = 0.126708
epoch: 15/100, loss = 0.124812
epoch: 16/100, loss = 0.122952
epoch: 17/100, loss = 0.121213
epoch: 18/100, loss = 0.119539
epoch: 19/100, loss = 0.117990
epoch: 20/100, loss = 0.116509
epoch: 21/100, loss = 0.115144
epoch: 22/100, loss = 0.113852
epoch: 23/100, loss = 0.112698
epoch: 24/100, loss = 0.111614
epoch: 25/100, loss = 0.110626
epoch: 26/100, loss = 0.109701
epoch: 27/100, loss = 0.108924
epoch: 28/100, loss = 0.108182
epoch: 29/100, loss = 0.107529
epoch: 30/100, loss = 0.106920
epoch: 31/100, loss = 0.106393
epoch: 32/100, loss = 0.105873
epoch: 33/100, lo